# Analysis

In [46]:
import json

with open('diffs.json', 'r') as f:
    diffs = json.load(f)

with open('codes.json', 'r') as cf:
    codes = json.load(cf)

## Dataset

In [47]:
TOTAL = 538
retrieved = len(diffs)
with_saving = len([d for d in diffs if 'saving' in d['existing_codes']])
print('retrieved commits:', f'{retrieved}/{TOTAL}', f'({retrieved/TOTAL*100:.2f}%)')
print('originally coded with `saving`:', f'{with_saving}/{retrieved}', f'({with_saving/retrieved*100:.2f}%)')

n_processed = sum(len(d['codes']) != 0 for d in diffs)
print('processed commits:', f'{n_processed}/{retrieved}', f'({n_processed/retrieved*100:.2f}%)')
n_processed_with_saving = sum(len(d['codes']) != 0 and 'saving' in d['existing_codes'] for d in diffs)
print('processed commits coded with `saving`:', f'{n_processed_with_saving}/{with_saving}', f'({n_processed_with_saving/with_saving*100:.2f}%)')

avg_files = sum(len(d['files']) for d in diffs) / len(diffs)
print('files affected (avg):', f'{avg_files:.2f}')

retrieved commits: 499/538 (92.75%)
originally coded with `saving`: 342/499 (68.54%)
processed commits: 452/499 (90.58%)
processed commits coded with `saving`: 322/342 (94.15%)
files affected (avg): 4.42


## Codes

In [48]:
import polars as pl

df = pl.DataFrame(codes) \
        .with_columns(count=sum(pl.col('name').is_in(d['codes']) for d in diffs)) \
        .with_columns(percentage=pl.col('count') / n_processed * 100) \
        .sort(by=pl.col('count'), descending=True)

print(df)

shape: (112, 4)
┌───────────────────────────────────┬───────────────────────────────────┬───────┬────────────┐
│ name                              ┆ description                       ┆ count ┆ percentage │
│ ---                               ┆ ---                               ┆ ---   ┆ ---        │
│ str                               ┆ str                               ┆ i32   ┆ f64        │
╞═══════════════════════════════════╪═══════════════════════════════════╪═══════╪════════════╡
│ aws                               ┆ Specific to Amazon Web Services   ┆ 328   ┆ 72.566372  │
│ not_relevant                      ┆ The commit contents do not refle… ┆ 158   ┆ 34.955752  │
│ cheaper_instance                  ┆ Use a cheaper compute instance    ┆ 70    ┆ 15.486726  │
│ gcp                               ┆ Specific to Google Cloud Platfor… ┆ 57    ┆ 12.610619  │
│ azure                             ┆ Specific to Azure                 ┆ 46    ┆ 10.176991  │
│ …                               

### Cloud-specific Codes

In [49]:
clouds = {'digitalocean', 'aws', 'gcp', 'azure', 'alicloud', 'scaleway', 'hcloud', 'ibm', 'oracle', 'ovh'}
print(df.filter(pl.col('name').is_in(clouds)).drop('description'))

shape: (10, 3)
┌──────────────┬───────┬────────────┐
│ name         ┆ count ┆ percentage │
│ ---          ┆ ---   ┆ ---        │
│ str          ┆ i32   ┆ f64        │
╞══════════════╪═══════╪════════════╡
│ aws          ┆ 328   ┆ 72.566372  │
│ gcp          ┆ 57    ┆ 12.610619  │
│ azure        ┆ 46    ┆ 10.176991  │
│ digitalocean ┆ 12    ┆ 2.654867   │
│ scaleway     ┆ 2     ┆ 0.442478   │
│ ibm          ┆ 2     ┆ 0.442478   │
│ oracle       ┆ 2     ┆ 0.442478   │
│ alicloud     ┆ 1     ┆ 0.221239   │
│ hcloud       ┆ 1     ┆ 0.221239   │
│ ovh          ┆ 1     ┆ 0.221239   │
└──────────────┴───────┴────────────┘


### Undefined codes

List codes which are attached to commits but not defined in `codes.json`.

In [50]:
listed_codes = set(c['name'] for c in codes)
for d in diffs:
    for c in d['codes']:
        if c not in listed_codes:
            print(f'{c} is not listed')